In [1]:
import pandas as pd
import numpy as np
import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display
import time
import matplotlib as mpl
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
mpl.rcParams['agg.path.chunksize'] = 10000

In [2]:
df = pd.read_csv(r'Data/NORMAL_DATA_RFM.csv')

In [15]:
df

,DOCNUM_ID,PEOPLE,STKCOD,DOCNUM,DOCDAT,TRNQTY,STKDES,NETVAL
0,1,บ-009,T1956015ASSURGY,RN1509170006,9/17/2015,5.0,ยางนอก GY 195/60-15 ASSUR,8500.00
1,2,บ-009,T1856015TZ700FS,RR170228001,3/1/2017,4.0,ยางนอก FS 185/60-15 TZ700,5981.31
2,3,บ-009,T1856015TZ700FS,RR170227001,3/1/2017,2.0,ยางนอก FS 185/60-15 TZ700,2990.65
3,4,บ-009,T1856015TZ700FS,RN170227004,3/3/2017,4.0,ยางนอก FS 185/60-15 FS100,5800.00
4,5,บ-009,T2453520K452FK,RN170727002,7/28/2017,12.0,ยางนอก FK 245/35-20 FK452,43200.00
...,...,...,...,...,...,...,...,...
13903,4902,บต-2573,M-SERVICE-01,HN1901092,2/1/2019,1.0,ค่าบริการ,650.00
13904,4902,บต-2573,M-08-02-0002,HN1901092,2/1/2019,1.0,ตั้งศูนย์,400.00
13905,4902,บต-2573,S-LAMP,HN1901092,2/1/2019,1.0,หลอดไฟหน้า-ขวา,220.00
13906,4902,บต-2573,B-3K-VS120R,HN1901092,2/1/2019,1.0,แบตเตอรี่ 3K VS120R:รถยนต์ 95D31R,3500.00


# Function that returns the size of an object in MB

In [2]:
def size(obj):
    return "{0:.2f} MB".format(sys.getsizeof(obj) / (1000 * 1000))

### Part 1:  Data Preparation

In [6]:
df_re=df.reindex(columns= [ 'DOCNUM_ID','DOCNUM', 'TRNQTY','STKDES','STKCOD']) 
print('orders -- dimensions: {0};   size: {1}'.format(df_re.shape, size(df_re)))
df_re

orders -- dimensions: (13908, 5);   size: 3.76 MB


,DOCNUM_ID,DOCNUM,TRNQTY,STKDES,STKCOD
0,1,RN1509170006,5.0,ยางนอก GY 195/60-15 ASSUR,T1956015ASSURGY
1,2,RR170228001,4.0,ยางนอก FS 185/60-15 TZ700,T1856015TZ700FS
2,3,RR170227001,2.0,ยางนอก FS 185/60-15 TZ700,T1856015TZ700FS
3,4,RN170227004,4.0,ยางนอก FS 185/60-15 FS100,T1856015TZ700FS
4,5,RN170727002,12.0,ยางนอก FK 245/35-20 FK452,T2453520K452FK
...,...,...,...,...,...
13903,4902,HN1901092,1.0,ค่าบริการ,M-SERVICE-01
13904,4902,HN1901092,1.0,ตั้งศูนย์,M-08-02-0002
13905,4902,HN1901092,1.0,หลอดไฟหน้า-ขวา,S-LAMP
13906,4902,HN1901092,1.0,แบตเตอรี่ 3K VS120R:รถยนต์ 95D31R,B-3K-VS120R


### Part 2: Association Rules Function

In [3]:
df.dropna(axis=0, subset=['DOCNUM'], inplace=True)
df['DOCNUM'] = df['DOCNUM'].astype('str')
df = df[~df['DOCNUM'].str.contains('C')]
df

,DOCNUM_ID,PEOPLE,STKCOD,DOCNUM,DOCDAT,TRNQTY,STKDES,NETVAL
0,1,บ-009,T1956015ASSURGY,RN1509170006,9/17/2015,5.0,ยางนอก GY 195/60-15 ASSUR,8500.00
1,2,บ-009,T1856015TZ700FS,RR170228001,3/1/2017,4.0,ยางนอก FS 185/60-15 TZ700,5981.31
2,3,บ-009,T1856015TZ700FS,RR170227001,3/1/2017,2.0,ยางนอก FS 185/60-15 TZ700,2990.65
3,4,บ-009,T1856015TZ700FS,RN170227004,3/3/2017,4.0,ยางนอก FS 185/60-15 FS100,5800.00
4,5,บ-009,T2453520K452FK,RN170727002,7/28/2017,12.0,ยางนอก FK 245/35-20 FK452,43200.00
...,...,...,...,...,...,...,...,...
13903,4902,บต-2573,M-SERVICE-01,HN1901092,2/1/2019,1.0,ค่าบริการ,650.00
13904,4902,บต-2573,M-08-02-0002,HN1901092,2/1/2019,1.0,ตั้งศูนย์,400.00
13905,4902,บต-2573,S-LAMP,HN1901092,2/1/2019,1.0,หลอดไฟหน้า-ขวา,220.00
13906,4902,บต-2573,B-3K-VS120R,HN1901092,2/1/2019,1.0,แบตเตอรี่ 3K VS120R:รถยนต์ 95D31R,3500.00


In [35]:
basket = (df.groupby(['DOCNUM', 'STKCOD'])['TRNQTY']
          .sum().unstack().reset_index().fillna(0)
          .set_index('DOCNUM'))


In [36]:
basket.to_csv(r'data/BASKET.csv', encoding='utf-8')

In [37]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [44]:
basket_sets

STKCOD,0W20MOBIL,0W40MOBIL1,0W40PERTAMINA,10W40PERTAMINI,14-10-02-001,14-10-09-01,15W40DELVACMOBIL,15W40MOBIL,15W40TOTAL,28697,...,ค-040,น้ำมันดำ,น้ำมันเครื่อง,ยางตัน 300-15,ยางตัน 700-12,ร-001,ล-001,ล-002,ลิ้นยาง15BS00,เจียรไฟวิน
DOCNUM,,,,,,,,,,,,,,,,,,,,,
GR171123001,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GR180125001,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GR180317001,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GR180317002,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GR180319001,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RR190124008,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RR190125001,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
RR190125003,0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
baseket_set_notnull = basket_sets.dropna()

In [59]:
# Build up the frequent items
frequent_itemsets = apriori(baseket_set_notnull, min_support=0.05, use_colnames=True)
frequent_itemsets

c:\Users\TIDE\AppData\Local\Programs\Python\Python38\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.131509,(53-04-07-00)
1,0.056565,(M-03-01-1516)
2,0.082499,(M-04-01-0003)
3,0.069634,(M-08-02-0002)
4,0.067797,(M-08-02-0003)
5,0.195630,(M-SERVICE-01)
6,0.132939,(S555)
7,0.066980,"(M-08-02-0003, M-04-01-0003)"
8,0.055953,"(M-SERVICE-01, S555)"


In [60]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(M-08-02-0003),(M-04-01-0003),0.067797,0.082499,0.066980,0.987952,11.975248,0.061387,76.152542
1,(M-04-01-0003),(M-08-02-0003),0.082499,0.067797,0.066980,0.811881,11.975248,0.061387,4.955397
2,(M-SERVICE-01),(S555),0.195630,0.132939,0.055953,0.286013,2.151464,0.029946,1.214393
3,(S555),(M-SERVICE-01),0.132939,0.195630,0.055953,0.420891,2.151464,0.029946,1.388978
